# Delaware income tax

Delaware has a single core rate structure that does not vary by filing status.
It also has other exemptions, deductions, and credits, including a two-tier federal Earned Income Tax Credit match: filers can claim either a refundable or non-refundable EITC match, but not both.

## Examples

### Full income tax

In [6]:
from policyengine_us import IndividualSim
import pandas as pd
import plotly.express as px

LIGHT_GRAY = "#F5F5F5"
GRAY = "#BDBDBD"
BLUE = "#5091cc"
LIGHT_BLUE = "lightblue"
DARK_BLUE = "darkblue"

COLOR_MAP = {"0": GRAY, "1": LIGHT_BLUE, "2": BLUE, "3": DARK_BLUE}


def make_tax(adults, children):
    sim = IndividualSim(year=2023)
    sim.add_person(name="head", age=25, rent=12_000)
    members = ["head"]
    if adults == 2:
        sim.add_person(name="spouse")
        members += ["spouse"]
    for i in range(children):
        child = "child{}".format(i)
        sim.add_person(name=child, age=6)
        members += [child]
    sim.add_tax_unit(name="tax_unit", members=members)
    sim.add_spm_unit(name="spm_unit", members=members)
    sim.add_household(name="household", members=members, state_code="DE")
    sim.vary("employment_income", max=100_000, step=100)
    return pd.DataFrame(
        dict(
            employment_income=sim.calc("employment_income")[0],
            de_income_tax=sim.calc("de_income_tax")[0].round(),
            de_eitc=sim.calc("de_eitc")[0].round(),
            de_refundable_eitc=sim.calc("de_refundable_eitc")[0].round(),
            de_non_refundable_eitc=sim.calc("de_non_refundable_eitc")[
                0
            ].round(),
            mtr=sim.deriv(
                "de_income_tax", "employment_income", wrt_target="head"
            ),
            adults=adults,
            children=str(children),
        )
    )


# Make a table of state taxes for different numbers of adults and children.
l = []
for adults in range(1, 3):
    for children in range(0, 4):
        l.append(make_tax(adults, children))

df = pd.concat(l)

LABELS = dict(
    employment_income="Employment income",
    de_income_tax="Delaware income tax",
    mtr="State marginal tax rate",
    adults="Adults",
    children="Children",
    de_eitc="Delaware EITC",
    de_refundable_eitc="Delaware refundable EITC",
    de_non_refundable_eitc="Delaware non-refundable EITC",
)

fig = px.line(
    df,
    "employment_income",
    "de_income_tax",
    color="children",
    animation_frame="adults",
    labels=LABELS,
    title="Delaware state income tax",
    color_discrete_map=COLOR_MAP,
)
fig.update_layout(
    xaxis_tickformat="$,",
    yaxis_tickformat="$,",
    plot_bgcolor="white",
    xaxis_gridcolor=LIGHT_GRAY,
    yaxis_gridcolor=LIGHT_GRAY,
)
fig.show()

Core marginal tax rates span from zero to five percent, but when including the EITC, they range from -13.5% to 12.3%, depending on income and household structure.

In [7]:
fig = px.line(
    df,
    "employment_income",
    "mtr",
    color="children",
    animation_frame="adults",
    labels=LABELS,
    title="Delaware state income tax marginal tax rate",
    color_discrete_map=COLOR_MAP,
)
fig.update_layout(
    xaxis_tickformat="$,",
    yaxis_tickformat=".1%",
    plot_bgcolor="white",
    xaxis_gridcolor=LIGHT_GRAY,
    yaxis_gridcolor=LIGHT_GRAY,
)
fig.show()

In [8]:
df

,employment_income,de_income_tax,de_eitc,de_refundable_eitc,de_non_refundable_eitc,mtr,adults,children
0,0.0,0.0,0.0,0.0,0.0,-0.003443,1,0
1,100.0,-0.0,0.0,0.0,0.0,-0.003443,1,0
2,200.0,-1.0,1.0,1.0,0.0,-0.003443,1,0
3,300.0,-1.0,1.0,1.0,0.0,-0.003442,1,0
4,400.0,-1.0,1.0,1.0,0.0,-0.003443,1,0
...,...,...,...,...,...,...,...,...
996,99600.0,0.0,0.0,0.0,0.0,0.000000,2,3
997,99700.0,0.0,0.0,0.0,0.0,0.000000,2,3
998,99800.0,0.0,0.0,0.0,0.0,0.000000,2,3
999,99900.0,0.0,0.0,0.0,0.0,0.000000,2,3


### Delaware EITC

Massachusetts matches 30 percent of a filer's federal Earned Income Tax Credit.

In [13]:
fig = px.line(
    # Filter to single person with no children
    df[(df.adults == 1) & (df.children == "3")],
    "employment_income",
    ["de_refundable_eitc", "de_non_refundable_eitc"],
    labels=LABELS,
    title="Delaware EITC for a single person with no children",
    color_discrete_map=COLOR_MAP,
)
fig.update_layout(
    xaxis_tickformat="$,",
    yaxis_tickformat="$,",
    plot_bgcolor="white",
    xaxis_gridcolor=LIGHT_GRAY,
    yaxis_gridcolor=LIGHT_GRAY,
)
fig.show()

### Massachusetts Limited Income Credit

See [**Schedule NTS-L-NR/PY** No Tax Status and Limited Income Credit](https://www.mass.gov/doc/2021-schedule-nts-l-nrpy-no-tax-status-and-limited-income-credit/download).

In [ ]:
fig = px.line(
    df,
    "employment_income",
    "ma_limited_income_tax_credit",
    color="children",
    animation_frame="adults",
    labels=LABELS,
    title="Massachusetts Limited Income Credit",
    color_discrete_map=COLOR_MAP,
)
fig.update_layout(
    xaxis_tickformat="$,",
    yaxis_tickformat="$,",
    yaxis_range=[0, df.ma_limited_income_tax_credit.max()],
    plot_bgcolor="white",
    xaxis_gridcolor=LIGHT_GRAY,
    yaxis_gridcolor=LIGHT_GRAY,
)
set_bounds(fig, df.employment_income, df.ma_limited_income_tax_credit)
fig.show()